In [19]:
%cd ..

/home/ltorres/leo/tesis/cloud-classification


In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
from torchvision.io import read_image
import glob
import os
import math


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from random import sample
import torch
import torch.nn as nn

In [22]:
from src.dataset import GCD
from src import config
from src import engine_gnn as engine
from src import utils
from src import sweeps

from src.models.graph_nets import GraphConvGNN, GATConvGNN

In [23]:
from sklearn.metrics import accuracy_score

In [24]:
import wandb

In [25]:
wandb.login()

True

In [26]:
device = config.DEVICE

### Sweep config

In [27]:
sweep_config = {
    'method': 'random'
    }

In [28]:
metric = {
    'name': 'test_accuracy',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric

In [29]:
# parameters_dict = {
#     'optimizer': {
#         'values': ['adam', 'sgd', 'nadam']
#         },
#     'criterion': {
#         'values': ['cross_entropy', 'loge']
#         },
#     'hidden_dim': {
#         'values': [128, 256, 512, 1024]
#         },
#     'num_hidden': {
#         'values': [2,3,4]
#         },
#     'num_heads': {
#           'values': [1,2,4]
#         },
#     'similarity_threshold': {
#           'values': [0.5, 0.6, 0.7, 0.75, 0.8, 0.9]
#         },
#     }

# sweep_config['parameters'] = parameters_dict

In [30]:
parameters_dict = {
    'optimizer': {
        'values': ['sgd', 'nadam']
        },
    'criterion': {
        'values': ['cross_entropy', 'loge']
        },
    'hidden_dim': {
        'values': [512, 1024]
        },
    'num_hidden': {
        'values': [2,3,4]
        },
    'num_heads': {
          'values': [1,4]
        },
    'similarity_threshold': {
          'values': [0.65, 0.7, 0.75, 0.8]
        },
    }

sweep_config['parameters'] = parameters_dict

In [31]:
parameters_dict.update({
    'epochs': {
        'value': config.EPOCHS,
        },
    'device': {
        'value': device,
        },
    'classes': {
        'value': config.GCD_CLASSES,
        },
    })

In [32]:
parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 5e-6,
        'max': 3e-5
      },
    'batch_size': {
        # integers between 16 and 32
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform',
        'q': 1,
        'min': math.log(16),
        'max': math.log(36),
      }
    })

In [33]:
sweep_config

{'method': 'random',
 'metric': {'name': 'test_accuracy', 'goal': 'maximize'},
 'parameters': {'optimizer': {'values': ['sgd', 'nadam']},
  'criterion': {'values': ['cross_entropy', 'loge']},
  'hidden_dim': {'values': [512, 1024]},
  'num_hidden': {'values': [2, 3, 4]},
  'num_heads': {'values': [1, 4]},
  'similarity_threshold': {'values': [0.65, 0.7, 0.75, 0.8]},
  'epochs': {'value': 30},
  'device': {'value': 'cuda:2'},
  'classes': {'value': ['1_cumulus',
    '2_altocumulus',
    '3_cirrus',
    '4_clearsky',
    '5_stratocumulus',
    '6_cumulonimbus',
    '7_mixed']},
  'learning_rate': {'distribution': 'uniform', 'min': 5e-06, 'max': 3e-05},
  'batch_size': {'distribution': 'q_log_uniform',
   'q': 1,
   'min': 2.772588722239781,
   'max': 3.58351893845611}}}

In [16]:
sweep_id = wandb.sweep(sweep_config, project="cloud classification")

Create sweep with ID: bwpa1fhk
Sweep URL: https://wandb.ai/leonardtd/cloud%20classification/sweeps/bwpa1fhk


---

In [17]:
def train(config=None):

#Init new run
    with wandb.init(config=config):

        config = wandb.config
        train_loader, test_loader, augmentation_loaders = utils.build_dataset_gcd(config.batch_size)

        model = utils.build_model_gatconv(
                                            7, #GCD num classes
                                            config.hidden_dim,
                                            config.num_hidden,
                                            config.num_heads,
                                            config.similarity_threshold,
                                            config.device
                                           )

        optimizer = utils.build_optimizer(config.optimizer, model, config.learning_rate)
        criterion = utils.build_criterion(config.criterion)

        for e in range(config.epochs):
            ### TRAIN DATASET
                preds, targets, loss = engine.train_fn(model, 
                                       train_loader, 
                                       criterion, 
                                       optimizer, 
                                       config.device)
                
                train_acc = accuracy_score(targets, preds)

                ### AUGMENTATION IMAGES
                for _loader in augmentation_loaders:
                    engine.forward_backward_pass(model, 
                                                 _loader, 
                                                 criterion, 
                                                 optimizer, 
                                                 device=config.device)

                test_preds, test_targets, test_loss = engine.eval_fn(model, 
                                           test_loader, 
                                           criterion, 
                                           device=config.device)
                test_acc = accuracy_score(test_targets, test_preds)


                print("EPOCH {}: Train acc: {:.2%} Train Loss: {:.4f} Test acc: {:.2%} Test Loss: {:.4f}".format(
                    e+1,
                    train_acc,
                    loss,
                    test_acc,
                    test_loss
                ))

                metrics = {
                            "train/train_loss": loss,
                            "train/train_accuracy": train_acc,
                            "test/test_loss": test_loss,
                            "test/test_accuracy": test_acc,
                          }

                wandb.log(metrics)
                wandb.log({"test_accuracy": test_acc, "epoch": e+1})
            
        wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None, 
                                                            preds=test_preds,
                                                            y_true=test_targets,
                                                            class_names=config.classes)})

---

In [ ]:
wandb.agent(sweep_id, train, count=10)

wandb: Agent Starting Run: xplu2su7 with config:
wandb: 	batch_size: 27
wandb: 	classes: ['1_cumulus', '2_altocumulus', '3_cirrus', '4_clearsky', '5_stratocumulus', '6_cumulonimbus', '7_mixed']
wandb: 	criterion: cross_entropy
wandb: 	device: cuda:2
wandb: 	epochs: 30
wandb: 	hidden_dim: 512
wandb: 	learning_rate: 2.8292267808489245e-05
wandb: 	num_heads: 1
wandb: 	num_hidden: 2
wandb: 	optimizer: nadam
wandb: 	similarity_threshold: 0.65


100%|██████████| 334/334 [00:18<00:00, 17.99it/s]


EPOCH 1: Train acc: 81.29% Train Loss: 1.0310 Test acc: 79.52% Test Loss: 1.4211


100%|██████████| 334/334 [00:18<00:00, 18.17it/s]


EPOCH 2: Train acc: 94.27% Train Loss: 0.3038 Test acc: 73.94% Test Loss: 1.7912


...

100%|██████████| 291/291 [00:18<00:00, 15.89it/s]


EPOCH 21: Train acc: 99.89% Train Loss: 0.0093 Test acc: 77.46% Test Loss: 2.3892


100%|██████████| 291/291 [00:18<00:00, 15.87it/s]


EPOCH 22: Train acc: 99.93% Train Loss: 0.0080 Test acc: 78.77% Test Loss: 2.3397


 85%|████████▌ | 276/323 [00:53<00:09,  5.19it/s]